In [1]:
import numpy as np
import numba
import time
import concurrent.futures
from numpy.random import Generator, PCG64DXSM, SeedSequence
from math import floor, ceil
from sys import float_info
np.set_printoptions(suppress=True)
np.set_printoptions(threshold=np.inf)
from FNs_groupID_0001_v0008 import gid0001_nsteps, population_array, id_sigs_fn, connections_init

In [2]:
runs = 10 #change this value to 100 if you want to confirm that the code is faster with 10 threads than 20, but it wil take 10 times longer to run
runlength = 10**5

sg = SeedSequence()
rgs = numba.typed.List([Generator(PCG64DXSM(s)) for s in sg.spawn(runs)])

thrds = 10 #number of threads that will be used

population_size = 10**3
levels_of_id = 4
branches_per_lvl = np.array([1, 2, 3, 2])
num_signals = np.prod(branches_per_lvl) #setting the number of signals equal to the number of unique identities
inertia = 100 #how many balls of each type are initially in the urns
rein_per_lvl = np.array([1, 2, 3, 4]) #payoff value for coordinating on each level, need array for numba to be happy
base_connection_weights = np.array([1, 2, 3, 4]) #base values for function determining likelihood of agent interacting with another of the same type
homophily_factor = 1
# percent_agents_per_lvl = [[1], [.1, .9], [[.3, .3, .4], [.9, .05, .05]], [[[.5, .5], [.5, .5], [.5, .5]], [[.5, .5], [.5, .5], [.5, .5]]]]
percent_agents_per_lvl = [[1], [.1, .9], [.3, .3, .4, .9, .05, .05], [.5, .5, .5, .5, .5, .5, .5, .5, .5, .5, .5, .5]]
epsilon = float_info.epsilon #add this value to random uniform distribution to get (0, 1] instead of [0, 1)


In [3]:
start = time.perf_counter()
# initializing some arrays that are fixed for all simulations
parent_size, popid = population_array(population_size, levels_of_id, branches_per_lvl, percent_agents_per_lvl)
print(parent_size)
idsigs = id_sigs_fn(popid, population_size, levels_of_id, branches_per_lvl)
conweights = connections_init(population_size, popid, base_connection_weights, homophily_factor)
# okay well this isn't fixed but its the same initial values for all simulations
agsigweights = inertia*np.ones([population_size, levels_of_id])

final_mp = np.zeros((runs, popid[-1][-1]+1, levels_of_id), dtype=np.int64)

inputs = []

with concurrent.futures.ProcessPoolExecutor(max_workers=thrds) as executor:
    future_to_gid_0001 = {executor.submit(gid0001_nsteps, idsigs, rein_per_lvl, conweights, agsigweights, popid, levels_of_id, population_size, epsilon, rgs[r], runlength, r): inputs for r in range(runs)}
    for future in concurrent.futures.as_completed(future_to_gid_0001):
        inputs = future_to_gid_0001[future]
        try:
            data0, data1 = future.result()
        except Exception as exc:
            print(f'generated an exception: y? and {exc}')
        else:
            final_mp[data1] = data0
            
final_mp = np.asarray(final_mp)
final_avg = np.mean(final_mp, axis=0)
print(final_avg)

finish = time.perf_counter()
print(f'Finished in {round(finish-start,0)/60} minutes')

[[1000], [100, 900], [30, 30, 40, 810, 45, 45], [15, 15, 15, 15, 20, 20, 405, 405, 23, 22, 23, 22]]
[[  0.  15.   0.   0.]
 [  0.  15.   0.   0.]
 [  0.  15.   0.   0.]
 [  0.  15.   0.   0.]
 [  0.  20.   0.   0.]
 [  0.  20.   0.   0.]
 [  0.   0. 405.   0.]
 [  0.   0. 405.   0.]
 [  0.  23.   0.   0.]
 [  0.  22.   0.   0.]
 [  0.  23.   0.   0.]
 [  0.  22.   0.   0.]]
Finished in 7.35 minutes
